In [1]:
import findspark
findspark.init('/home/akshay/spark-3.5.1-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_tutorials_1").getOrCreate()

24/06/29 18:04:20 WARN Utils: Your hostname, akshay-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/29 18:04:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/29 18:04:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/29 18:04:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/29 18:04:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
data = [
    ("Alice", '{"address":{"city": "New York", "state": "NY"}, "gender": "female"}'),
    ("Bob", '{"address":{"city": "Los Angeles", "state": "CA"}, "gender": "male"}'),
    ("Charlie", '{"address":{"city": "Chicago", "state": "IL"}, "gender": "male"}'),
    ("Diana", '{"address":{"city": "Houston", "state": "TX"}, "gender": "female"}'),
    ("Eve", '{"address":{"city": "Miami", "state": "FL"}, "gender": "female"}')
]


In [3]:
schema = ["name","props"]

In [4]:
df = spark.createDataFrame(data=data, schema=schema)

In [5]:
df.show(truncate=False)

+-------+--------------------------------------------------------------------+
|name   |props                                                               |
+-------+--------------------------------------------------------------------+
|Alice  |{"address":{"city": "New York", "state": "NY"}, "gender": "female"} |
|Bob    |{"address":{"city": "Los Angeles", "state": "CA"}, "gender": "male"}|
|Charlie|{"address":{"city": "Chicago", "state": "IL"}, "gender": "male"}    |
|Diana  |{"address":{"city": "Houston", "state": "TX"}, "gender": "female"}  |
|Eve    |{"address":{"city": "Miami", "state": "FL"}, "gender": "female"}    |
+-------+--------------------------------------------------------------------+



In [7]:
from pyspark.sql.functions import get_json_object

In [8]:
df1 = df.withColumn("gender", get_json_object("props","$.gender"))


In [9]:
df1.show(truncate=False)

+-------+--------------------------------------------------------------------+------+
|name   |props                                                               |gender|
+-------+--------------------------------------------------------------------+------+
|Alice  |{"address":{"city": "New York", "state": "NY"}, "gender": "female"} |female|
|Bob    |{"address":{"city": "Los Angeles", "state": "CA"}, "gender": "male"}|male  |
|Charlie|{"address":{"city": "Chicago", "state": "IL"}, "gender": "male"}    |male  |
|Diana  |{"address":{"city": "Houston", "state": "TX"}, "gender": "female"}  |female|
|Eve    |{"address":{"city": "Miami", "state": "FL"}, "gender": "female"}    |female|
+-------+--------------------------------------------------------------------+------+



In [10]:
df2 = df1.withColumn("city",get_json_object("props", "$.address.city"))
df2.show(truncate=False)

+-------+--------------------------------------------------------------------+------+-----------+
|name   |props                                                               |gender|city       |
+-------+--------------------------------------------------------------------+------+-----------+
|Alice  |{"address":{"city": "New York", "state": "NY"}, "gender": "female"} |female|New York   |
|Bob    |{"address":{"city": "Los Angeles", "state": "CA"}, "gender": "male"}|male  |Los Angeles|
|Charlie|{"address":{"city": "Chicago", "state": "IL"}, "gender": "male"}    |male  |Chicago    |
|Diana  |{"address":{"city": "Houston", "state": "TX"}, "gender": "female"}  |female|Houston    |
|Eve    |{"address":{"city": "Miami", "state": "FL"}, "gender": "female"}    |female|Miami      |
+-------+--------------------------------------------------------------------+------+-----------+



In [11]:
help(get_json_object)

Help on function get_json_object in module pyspark.sql.functions:

get_json_object(col: 'ColumnOrName', path: str) -> pyspark.sql.column.Column
    Extracts json object from a json string based on json `path` specified, and returns json string
    of the extracted json object. It will return null if the input json string is invalid.
    
    .. versionadded:: 1.6.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    col : :class:`~pyspark.sql.Column` or str
        string column in json format
    path : str
        path to the json object to extract
    
    Returns
    -------
    :class:`~pyspark.sql.Column`
        string representation of given JSON object value.
    
    Examples
    --------
    >>> data = [("1", '''{"f1": "value1", "f2": "value2"}'''), ("2", '''{"f1": "value12"}''')]
    >>> df = spark.createDataFrame(data, ("key", "jstring"))
    >>> df.select(df.key, get_json_object(df.jstring, '$.f1').alias("c0"), \
    .